In [2]:
!nvcc --version       # CUDA версия (должна быть 11.8)
!nvidia-smi           # Информация о GPU
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2  # cuDNN версия

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed Apr  2 18:40:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8       

In [3]:
# Обновите пакеты
!apt update && apt upgrade -y

# Установите компиляторы и зависимости
!apt install -y build-essential cmake unzip git
!apt install -y libopencv-dev  # Для OpenCV

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.3 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,802 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2

In [5]:
!rm -rf darknet  # Удаляет папку и всё её содержимое
# Клонируйте репозиторий
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

# Настройте Makefile (включите GPU, OpenCV, cuDNN)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# Для CUDA 11+ добавьте флаг совместимости (если нужно)
!sed -i 's/ARCH= -gencode arch=compute_30,code=sm_30/ARCH= -gencode arch=compute_75,code=sm_75/' Makefile

Cloning into 'darknet'...
remote: Enumerating objects: 15906, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15906 (delta 31), reused 30 (delta 17), pack-reused 15845 (from 2)
Receiving objects: 100% (15906/15906), 14.51 MiB | 22.68 MiB/s, done.
Resolving deltas: 100% (10697/10697), done.
/content/darknet/darknet


In [6]:
!echo "CUDA_PATH: $CUDA_HOME"
!echo "CUDNN_PATH: /usr/include/"

CUDA_PATH: 
CUDNN_PATH: /usr/include/


In [7]:
# Вручную задайте пути (пример)
!export CUDA_HOME=/usr/local/cuda
!export PATH=$CUDA_HOME/bin:$PATH
!export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH

In [8]:
# Очистите предыдущие сборки (если были)
!make clean

# Запустите компиляцию
!make -j8


rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/representation_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/go.o ./obj/batchnorm_layer.o ./obj/art.o ./obj/region_layer.o ./obj

In [9]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights  # Веса модели

--2025-04-02 18:54:20--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  23.9MB/s    in 1.4s    

2025-04-02 18:54:22 (23.9 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [ ]:
!mkdir data
%cd data

# Скачиваем изображения и аннотации COCO 2017
!wget http://images.cocodataset.org/zips/train2017.zip    # 19 ГБ (обучающие изображения)
!wget http://images.cocodataset.org/zips/val2017.zip      # 1 ГБ (валидационные)
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip  # Аннотации

# Распаковываем
!unzip train2017.zip
!unzip val2017.zip
!unzip annotations_trainval2017.zip

mkdir: cannot create directory ‘data’: File exists
/content/darknet/darknet/data
--2025-04-02 18:54:37--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.168.9, 3.5.22.221, 52.217.138.25, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.168.9|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip        92%[=================>  ]  16.72G  56.5MB/s    eta 24s    